In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("data/strings.csv")

In [3]:
df.shape
df

,String,Ref. Ten. (lbs),Swing Speed,Material,Gauge Nominal (mm),Gauge Acutal (mm),Stretch at 40 lbs (%),Stretch at 51 lbs (%),Stretch at 62 lbs (%),Actual Pre-impact Tension (lbs),...,Stiffness (lb/in),Static Loss lbs.,Stabilization Loss (lbs),Impact Loss (lbs).,Total Loss (lbs),Tension Loss (%),Energy Return (%),String/String COF,String/Ball COF,Spin Potential
0,Alien Black Diamond 16,51,Medium,Polyester,1.30,1.31,3.0,5.2,5.8,25.42,...,207.5,8.40,16.90,0.33,25.6,50.5,88.5,0.100,0.453,4.5
1,Alpha Gut 2000 16,51,Medium,Nylon,1.33,NaN,7.6,9.1,10.0,42.71,...,161.2,4.32,3.69,-0.02,7.9,15.7,96.4,0.161,0.506,3.1
2,Ashaway Liberty 16,51,Medium,Nylon,1.30,1.33,NaN,NaN,11.2,38.61,...,169.7,7.19,4.92,0.09,12.1,24.0,95.5,0.094,0.434,4.6
3,Ashaway Dynamite Soft 17,51,Medium,Nylon/Zyex,1.25,1.21,8.5,9.1,12.7,36.76,...,127.4,6.99,6.97,0.02,13.9,27.6,94.7,0.204,0.527,2.6
4,Ashaway MonoGut 16L,51,Medium,Polyester,1.27,1.28,1.8,3.0,4.2,27.45,...,228.0,8.16,15.11,0.28,23.6,46.4,89.7,0.106,0.366,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Yonex Poly Tour 130/16,51,Medium,Polyester,1.30,1.29,NaN,2.3,3.5,26.20,...,232.6,8.71,15.81,0.18,24.7,48.7,88.2,NaN,NaN,NaN
500,Yonex Poly Tour Spin G 1.25,51,Medium,Polyester,1.25,NaN,1.5,2.1,2.7,38.43,...,252.0,4.30,7.99,0.13,12.3,24.5,92.7,0.104,0.497,4.8
501,Yonex Poly Tour Pro Yellow 16L (1.25),51,Medium,Polyester,1.25,1.25,1.8,3.0,3.9,27.65,...,198.9,7.08,15.99,0.20,23.4,45.9,90.8,NaN,NaN,NaN
502,Yonex Tournament 80 Spin 15L-16,51,Medium,NaN,1.33,1.41,8.8,10.0,11.2,40.04,...,166.9,6.68,4.00,0.04,10.8,21.1,92.9,NaN,0.451,NaN


In [4]:
df.columns

Index(['String', 'Ref. Ten. (lbs)', 'Swing Speed', 'Material',
       'Gauge Nominal (mm)', 'Gauge Acutal (mm)', 'Stretch at 40 lbs (%)',
       'Stretch at 51 lbs (%)', 'Stretch at 62 lbs (%)',
       'Actual Pre-impact Tension (lbs)', 'Dwell Time (ms)', 'Deflection (mm)',
       'Tension Change (lbs)', 'Peak Tension (lbs)', 'Peak Perp. Force (lbs)',
       'Ave Perp. Force (lbs)', 'Stiffness (lb/in)', 'Static Loss lbs.',
       'Stabilization Loss (lbs)', 'Impact Loss (lbs).', 'Total Loss (lbs)',
       'Tension Loss (%)', 'Energy Return (%)', 'String/String COF',
       'String/Ball COF', 'Spin Potential'],
      dtype='object')

In [ ]:
df = df[['String', 'Swing Speed', 'Material',
       'Gauge Nominal (mm)', 'Gauge Acutal (mm)', 'Stretch at 40 lbs (%)',
       'Stretch at 51 lbs (%)', 'Stretch at 62 lbs (%)',
       'Actual Pre-impact Tension (lbs)', 'Dwell Time (ms)', 'Deflection (mm)',
       'Tension Change (lbs)', 'Peak Tension (lbs)', 'Peak Perp. Force (lbs)',
       'Ave Perp. Force (lbs)', 'Stiffness (lb/in)', 'Static Loss lbs.',
       'Stabilization Loss (lbs)', 'Impact Loss (lbs).', 'Total Loss (lbs)',
       'Tension Loss (%)', 'Energy Return (%)', 'String/String COF',
       'String/Ball COF', 'Spin Potential']]